In [2]:
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score

class XGBoostModel(BasePortfolioModel):
    def __init__(self, n_estimators=100, max_depth=3, learning_rate=0.1, optimize_hyperparams=False):
        self.optimize_hyperparams = optimize_hyperparams
        self.model = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                      learning_rate=learning_rate, objective='reg:squarederror')

    def fit(self, X_train, y_train):
        """Entrena el modelo y realiza tuning si está activado"""
        if self.optimize_hyperparams:
            def objective(trial):
                """Función objetivo para optimización de hiperparámetros"""
                params = {
                    'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                    'max_depth': trial.suggest_int('max_depth', 3, 10),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                    'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                    'gamma': trial.suggest_float('gamma', 0, 0.5),
                    'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
                    'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 2.0)
                }

                model = xgb.XGBRegressor(**params, objective='reg:squarederror')
                scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
                return scores.mean()

            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=30)

            best_params = study.best_params
            print("Mejores hiperparámetros encontrados:", best_params)

            # Crear modelo con los mejores hiperparámetros
            self.model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')

        # Entrenar el modelo final
        self.model.fit(X_train, y_train)
        print("XGBoost model fitted.")

    def predict(self, X):
        return self.model.predict(X)


/Users/adrianlopez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/adrianlopez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


NameError: name 'BasePortfolioModel' is not defined

In [1]:
# Cargar los datos
df = pd.read_csv('sampled_data.csv')

# Dividir entre X e y
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Separar en Train y Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo con tuning activado
xgb_model = XGBoostModel(optimize_hyperparams=True)
xgb_model.fit(X_train, y_train)

# Evaluar el modelo
xgb_model.evaluate(X_test, y_test)


NameError: name 'pd' is not defined